In [129]:
import VGG_Face
from deepface.commons import functions
from os import walk
import pandas as pd

from skimage.color import gray2rgb

In [130]:
img_w = 224
img_h = 224
img_gray = (1, img_w, img_h)
img_shape = (1, img_w, img_h, 3)

In [131]:
file_name = 'preprocessing_face.json'
path_test = './dataset/test/'

def face_verification(file_name, path_test):
    new_path_test = next(walk(path_test), (None, None, []))[2]
    result = []
    
    imgs_df = pd.read_csv('./preprocessing_face.csv')
    names_df = pd.read_csv('./names.csv')
     
    for img_test_name in new_path_test:
        flag = None
        name = 'Unknown'
        min = 2
        
        target_size = functions.find_target_size(model_name= 'VGG-Face')
        img_objs_test = functions.extract_faces( path_test + img_test_name,
                                                 target_size= target_size,
                                                 detector_backend= 'mtcnn',
                                                 grayscale=False,
                                                 enforce_detection=True,
                                                 align=True)
        img_test = img_objs_test[0][0]
        img_test = functions.normalize_input(img=img_test, normalization='base')
        
        for img, names in zip(imgs_df.values, names_df.values):
            img_res = img.reshape(img_gray)
            img_rgb_res = gray2rgb(img_res).reshape(img_shape)
            
            verify = VGG_Face.verify(img1 = img_test, 
                                    img2 = img_rgb_res,
                                    distance_metric= 'euclidean_l2',
                                    model_name= 'VGG-Face',
                                    )
                                        
            if verify['verified'] == True and verify['distance'] < min:
                name = names[0]
                min = verify['distance']
                flag = verify
                if min < 0.15:
                    break
                
        if min > 0.55:
            name = 'Unknown'
            
        result.append((img_test_name, name, flag['distance'], flag['time']))   
    return result 

In [132]:
result = face_verification(file_name, path_test)

1/1 [==============================] - 0s 22ms/step


In [133]:
for i in range(len(result)):
    print(f'{i + 1}.')
    print('\tFile: ', result[i][0])
    print('\tName: ', result[i][1])
    print('\tDistance L2: ', result[i][2])
    print('\tTime: ', result[i][3])

1.
	File:  Jung Hae In.jpg
	Name:  Unknown
	Distance L2:  0.6410702052603633
	Time:  0.34
2.
	File:  Lee Dong Wook.jpg
	Name:  Lee Dong Wook
	Distance L2:  0.47467468558274345
	Time:  0.34
